In [ ]:
# ===============================================
# 05_hubble_grid_scan.py  (for notebook use)
# ===============================================
import numpy as np
from scipy.integrate import quad
import matplotlib.pyplot as plt

# ---- constants -------------------------------------------------
c        = 299_792.458          # km s⁻¹
t_pl     = 5.391247e-44         # s
t0       = 4.35e17              # 13.8 Gyr in s
ln_ratio = np.log(t0 / t_pl)

Ω_r   = 9e-5
Ω_m0  = 0.30
H0_list    = [67.0, 70.0, 73.0]
gamma_list = [0.01, 0.05, 0.10]

# ---- helper functions ------------------------------------------
def Ez(z, H0, Ωm, γ):
    """Dimensionless H(z)/H0 with flux dark-energy."""
    op = np.tanh(γ * (ln_ratio - 1.5*np.log1p(z)))**2
    return np.sqrt(Ωm*(1+z)**3 + (1-Ωm)*op + Ω_r*(1+z)**4)

def dL(z, H0, Ωm, γ):
    """Luminosity distance in Mpc (simple trapezoid)."""
    zs = np.linspace(0, z, 200)
    integral = np.trapz(1/Ez(zs, H0, Ωm, γ), zs)
    return (c/H0) * (1+z) * integral

def rs_cmb(H0, Ωm, z_star=1100.0):
    """Sound horizon (ΛCDM H(z) for high-z)."""
    c_s = c / np.sqrt(3)
    integrand = lambda z: c_s / (
        H0*np.sqrt(Ω_r*(1+z)**4 + Ωm*(1+z)**3 + (1-Ωm-Ω_r))
    )
    rs, _ = quad(integrand, z_star, 1e5)
    return rs

# --- tiny synthetic SN data -------------------------------------
z_sn   = np.array([0.1, 0.5, 1.0, 1.5])
mu_obs = np.array([38.2, 42.3, 44.1, 45.3])  # mock values
mu_err = np.array([0.20, 0.20, 0.25, 0.30])
Moff   = -19.3                                # fixed zero-point

def mu_th(z, H0, Ωm, γ):
    return 5*np.log10([dL(zi, H0, Ωm, γ) for zi in z]) + 25 + Moff

def chi2_sn(H0, Ωm, γ):
    return np.sum(((mu_obs - mu_th(z_sn, H0, Ωm, γ)) / mu_err)**2)

# ---- grid scan -------------------------------------------------
rows = []
for H0 in H0_list:
    for γ in gamma_list:
        rows.append(dict(
            H0=H0,
            gamma=γ,
            chi2_sn=chi2_sn(H0, Ω_m0, γ),
            rs      =rs_cmb(H0, Ω_m0),
        ))
import pandas as pd
df = pd.DataFrame(rows)
df["Δr_s"] = df["rs"] - 144.9  # Planck reference (Mpc)

print(df.to_markdown(index=False))

# ---- H(z) comparison plot --------------------------------------
z_plot = np.linspace(0, 2, 200)
H_LCDM = 70.0 * np.sqrt(Ω_r*(1+z_plot)**4 + Ω_m0*(1+z_plot)**3 + (1-Ω_m0-Ω_r))
H_flux = 70.0 * Ez(z_plot, 70.0, Ω_m0, 0.05)

plt.figure(figsize=(8,5))
plt.plot(z_plot, H_LCDM, label="ΛCDM  (H₀=70)", lw=2)
plt.plot(z_plot, H_flux, "--", label="Flux  (H₀=70, γ=0.05)", lw=2)
plt.xlabel("Redshift  z"); plt.ylabel("H(z)  [km s⁻¹ Mpc⁻¹]")
plt.title("H(z) : ΛCDM vs. Quaternionic Flux")
plt.grid(alpha=.3, ls="--"); plt.legend()
plt.tight_layout(); plt.show()
